In [1]:
import warnings
warnings.filterwarnings("ignore")
import librosa 
from scipy.io import wavfile as wav
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 


In [2]:
# Class: English
filename = 'lang/en/_e4fgbYJsYM__U__S1---0228.380-0244.080.wav'
librosa_audio, librosa_sample_rate = librosa.load(filename) 
scipy_sample_rate, scipy_audio = wav.read(filename)
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio);

FileNotFoundError: [Errno 2] No such file or directory: 'lang/en/_e4fgbYJsYM__U__S1---0228.380-0244.080.wav'

In [3]:
ls

Phase#3_Data_Analysis_and_Preprocessing.ipynb
README.md


In [ ]:
# Class: Arabic
filename = 'lang/ar/_73h_mgRMGU__U__S0---0115.460-0124.760.wav'
librosa_audio, librosa_sample_rate = librosa.load(filename) 
scipy_sample_rate, scipy_audio = wav.read(filename)
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio);

## Get all audio files in Dataframe with its label (The language)

In [ ]:
Fname = []
# return all files as a list
for file in os.listdir(r'lang/en/'):
     # check the files which are end with specific extension
    if file.endswith(".wav"):
        # print path name of selected files
        Fname.append(file)

In [ ]:
dfEn = pd.DataFrame (Fname, columns = ['name'])
dfEn['class_name'] = 'English'
dfEn.head()

In [ ]:
dfEn.shape

In [ ]:
Fname = []
# return all files as a list
for file in os.listdir(r'lang/ar/'):
     # check the files which are end with specific extension
    if file.endswith(".wav"):
        # print path name of selected files
        Fname.append(file)
dfAr = pd.DataFrame (Fname, columns = ['name'])
dfAr['class_name'] = 'Arabic'
dfAr.head()

In [ ]:
dfAr.shape

## Extract Features for every file (Mel-Frequency Cepstral Coefficients (MFCC))

In [ ]:
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [ ]:
fulldatasetpath = 'lang/en/'

metadata = dfEn.head(10) # For now we only got 10 aufios

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath),str(row["name"]))
    class_label = row["class_name"]
    data = extract_features(file_name)
    features.append([data, class_label])
    
# Convert into a Panda dataframe 
featuresdfEn = pd.DataFrame(features, columns=['feature','class_label'])
print('Finished feature extraction from ', len(featuresdfEn), ' English files') 

In [ ]:
featuresdfEn.head()

In [ ]:
fulldatasetpath = 'lang/ar/'

metadata = dfAr.head(10) # For now we only got 10 aufios

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath),str(row["name"]))
    class_label = row["class_name"]
    data = extract_features(file_name)
    features.append([data, class_label])
    
# Convert into a Panda dataframe 
featuresdfAr = pd.DataFrame(features, columns=['feature','class_label'])
print('Finished feature extraction from ', len(featuresdfAr), ' Arabic files') 
featuresdfAr.head()

## Prepare data for training

Merge all dataframe for all Languages.
Use LabelEncoder to encode the categorical text data (Languages), ...

In [ ]:
featuresdf = pd.concat([featuresdfEn, featuresdfAr], ignore_index=True)

In [ ]:
featuresdf

In [ ]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())

# Encode the classification labels
ohe = OneHotEncoder()
# yy = ohe.fit_transform(y)
yy = ohe.fit_transform(featuresdf.class_label.values.reshape(-1,1)).toarray()

In [ ]:
yy

In [ ]:
ohe.categories_

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)